<a href="https://colab.research.google.com/github/GregB77/calor_run/blob/main/work_on_gpx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Convert GPX to CSV

In [9]:
# 1. Installation de la librairie gpxpy (nécessaire car non installée par défaut)
!pip install gpxpy pandas

import gpxpy
import pandas as pd
from google.colab import files
import io

print("--- ÉTAPE 1 : IMPORTATION ---")
print("Veuillez choisir votre fichier .gpx depuis votre ordinateur.")

# 2. Upload du fichier GPX
uploaded = files.upload()

# Récupération du nom du fichier (le premier fichier uploadé)
filename = next(iter(uploaded))

print(f"\nTraitement du fichier : {filename}...")

# 3. Parsing (Lecture) du fichier GPX
data_points = []

with open(filename, 'r') as gpx_file:
    gpx = gpxpy.parse(gpx_file)

    # On parcourt les traces (tracks), segments et points
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                # On ajoute les données dans une liste
                data_points.append({
                    'Latitude': point.latitude,
                    'Longitude': point.longitude,
                    'Altitude': point.elevation,
                    'Temps': point.time
                })

# 4. Création du DataFrame avec Pandas
df = pd.DataFrame(data_points)

print(f"\n✅ Succès : {len(df)} points géographiques ont été créés.")

# 5. Conversion en CSV et téléchargement
output_filename = filename.replace('.gpx', '.csv')
df.to_csv(output_filename, index=False)

print(f"\n--- TÉLÉCHARGEMENT ---")
print(f"Le fichier '{output_filename}' est en cours de téléchargement...")
files.download(output_filename)

--- ÉTAPE 1 : IMPORTATION ---
Veuillez choisir votre fichier .gpx depuis votre ordinateur.


Saving Calor Run 10km.gpx to Calor Run 10km.gpx

Traitement du fichier : Calor Run 10km.gpx...

✅ Succès : 460 points géographiques ont été créés.

--- TÉLÉCHARGEMENT ---
Le fichier 'Calor Run 10km.csv' est en cours de téléchargement...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Convert CSV to GPX

In [8]:
# 1. Installation de la librairie gpxpy
!pip install gpxpy pandas

import gpxpy
import gpxpy.gpx
import pandas as pd
from google.colab import files

print("--- ÉTAPE 1 : IMPORTATION ---")
print("Veuillez choisir votre fichier .csv (avec colonnes: lon, lat, ele, highway, surface)")

# 2. Upload du fichier CSV
uploaded = files.upload()
filename = next(iter(uploaded))

print(f"\nLecture du fichier : {filename}...")

# 3. Lecture du CSV avec Pandas
# On s'assure que les colonnes sont bien lues
df = pd.read_csv(filename)

# Vérification rapide des colonnes
required_cols = ['lat', 'lon']
if not all(col in df.columns for col in required_cols):
    print(f"ERREUR : Les colonnes obligatoires 'lat' et 'lon' sont manquantes.")
else:
    # 4. Création de l'objet GPX
    gpx = gpxpy.gpx.GPX()

    # Création d'une trace (Track)
    gpx_track = gpxpy.gpx.GPXTrack()
    gpx.tracks.append(gpx_track)

    # Création d'un segment
    gpx_segment = gpxpy.gpx.GPXTrackSegment()
    gpx_track.segments.append(gpx_segment)

    # 5. Boucle sur chaque ligne pour créer les points GPX
    for index, row in df.iterrows():
        # Création du point (Latitude, Longitude, Altitude)
        # On utilise .get() pour 'ele' au cas où la colonne serait vide ou manquante par erreur
        ele = row.get('ele', 0)

        point = gpxpy.gpx.GPXTrackPoint(
            latitude=row['lat'],
            longitude=row['lon'],
            elevation=ele
        )

        # Ajout des infos 'highway' et 'surface' dans la description du point
        # Cela permettra de voir ces infos en cliquant sur le point dans un lecteur GPX
        infos = []
        if 'highway' in row and pd.notna(row['highway']):
            infos.append(f"highway={row['highway']}")
        if 'surface' in row and pd.notna(row['surface']):
            infos.append(f"surface={row['surface']}")

        if infos:
            point.description = ", ".join(infos)

        gpx_segment.points.append(point)

    print(f"\n✅ Succès : {len(df)} points géographiques ont été créés.")

    # 6. Sauvegarde et Téléchargement
    output_filename = filename.replace('.csv', '.gpx')

    with open(output_filename, 'w') as f:
        f.write(gpx.to_xml())

    print(f"\n--- TÉLÉCHARGEMENT ---")
    print(f"Le fichier '{output_filename}' a été créé avec succès.")
    files.download(output_filename)

--- ÉTAPE 1 : IMPORTATION ---
Veuillez choisir votre fichier .csv (avec colonnes: lon, lat, ele, highway, surface)


Saving Calor Run 10km.csv to Calor Run 10km (1).csv

Lecture du fichier : Calor Run 10km (1).csv...
Conversion de 460 points en cours...

✅ Succès : 460 points géographiques ont été créés.

--- TÉLÉCHARGEMENT ---
Le fichier 'Calor Run 10km (1).gpx' a été créé avec succès.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Convert CSV to GEOJSON

In [7]:
# 1. Installation des librairies nécessaires
# geopandas n'est pas installé par défaut sur Colab
!pip install geopandas

import pandas as pd
import geopandas as gpd
from google.colab import files
import os

print("--- ÉTAPE 1 : IMPORTATION ---")
print("Veuillez choisir votre fichier .csv (avec colonnes: lon, lat, ele, highway, surface)")

# 2. Upload du fichier
uploaded = files.upload()
filename = next(iter(uploaded))

print(f"\nTraitement du fichier : {filename}...")

# 3. Lecture du CSV
df = pd.read_csv(filename)

# Vérification que les colonnes de coordonnées existent
if 'lon' not in df.columns or 'lat' not in df.columns:
    print("ERREUR : Les colonnes 'lon' et 'lat' sont introuvables dans le CSV.")
else:
    # 4. Conversion en GeoDataFrame
    # Cette étape transforme le tableau simple en objet géographique
    # gpd.points_from_xy crée la géométrie à partir de la longitude et latitude
    gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df['lon'], df['lat']),
        crs="EPSG:4326" # EPSG:4326 est le code standard pour les coordonnées GPS (WGS84)
    )

    # Note : Les colonnes 'ele', 'highway', 'surface' sont conservées automatiquement
    # et deviendront des "properties" dans le GeoJSON.

    print(f"\n✅ Succès : {len(gdf)} points géographiques ont été créés.")

    # 5. Export en GeoJSON
    output_filename = os.path.splitext(filename)[0] + ".geojson"
    gdf.to_file(output_filename, driver='GeoJSON')

    print("\n--- APERÇU DU RÉSULTAT ---")
    print(gdf.head())

    print(f"\n--- TÉLÉCHARGEMENT ---")
    print(f"Le fichier '{output_filename}' est prêt.")
    files.download(output_filename)

--- ÉTAPE 1 : IMPORTATION ---
Veuillez choisir votre fichier .csv (avec colonnes: lon, lat, ele, highway, surface)


Saving Calor Run 10km.csv to Calor Run 10km.csv

Traitement du fichier : Calor Run 10km.csv...

--- APERÇU DU RÉSULTAT ---
         lat       lon      ele highway  surface                  geometry
0  45.553319  5.003519  239.000   track  asphalt  POINT (5.00352 45.55332)
1  45.553299  5.003024  238.010   track  asphalt   POINT (5.00302 45.5533)
2  45.553069  5.001601  237.000    path    grass   POINT (5.0016 45.55307)
3  45.553668  5.001388  236.795    path    grass  POINT (5.00139 45.55367)
4  45.553762  5.001353  236.387    path    grass  POINT (5.00135 45.55376)

--- TÉLÉCHARGEMENT ---
Le fichier 'Calor Run 10km.geojson' est prêt.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Convert CSV to GEOJSON V2

In [14]:
# 1. Installation des librairies nécessaires
!pip install gpxpy

import gpxpy
import json
from google.colab import files
import os
import re

# --- FONCTION D'EXTRACTION SPÉCIFIQUE ---
def extract_gpx_data_for_custom_geojson(gpx_content):
    """
    Analyse le contenu GPX.
    Retourne la liste des coordonnées (LineString) et la liste des propriétés
    associées à chaque point (coordinateProperties).
    """
    coordinates = []
    coord_properties = []

    # Expression régulière pour trouver les motifs 'key=value' dans la description
    PATTERN = re.compile(r'(\w+)=([^,]+)')

    try:
        gpx = gpxpy.parse(gpx_content)

        # Nous allons traiter la première trace et le premier segment trouvé
        if not gpx.tracks or not gpx.tracks[0].segments:
            return None, None, "No track or segment found in GPX"

        gpx_track = gpx.tracks[0]
        gpx_segment = gpx_track.segments[0]

        # Propriétés de la trace (pour le champ 'name')
        track_name = gpx_track.name if gpx_track.name else "Trace GPX"

        for point in gpx_segment.points:

            # 1. Coordonnées [lon, lat, ele]
            coord = [point.longitude, point.latitude]
            if point.elevation is not None:
                 coord.append(point.elevation) # Ajout de l'élévation (si présente)

            coordinates.append(coord)

            # 2. Propriétés spécifiques du point
            props = {}
            if point.description:
                matches = PATTERN.findall(point.description)

                for key, value in matches:
                    if key in ['highway', 'surface']:
                        props[key] = value.strip()

            # Ajout des propriétés (même si elles sont vides pour maintenir la correspondance)
            coord_properties.append(props)

    except Exception as e:
        print(f"ERREUR lors du parsing du fichier GPX : {e}")
        return None, None, str(e)

    return coordinates, coord_properties, track_name

# --- DÉBUT DU SCRIPT PRINCIPAL ---

print("--- ÉTAPE 1 : IMPORTATION ---")
print("Veuillez choisir votre fichier .gpx.")

# 2. Upload du fichier GPX
uploaded = files.upload()
filename = next(iter(uploaded))

print(f"\nTraitement du fichier : {filename}...")
gpx_file_content = uploaded[filename].decode('utf-8')

# 3. Extraction des données
coordinates, coord_properties, track_name = extract_gpx_data_for_custom_geojson(gpx_file_content)

if coordinates is None:
    print(f"Erreur d'extraction : {track_name}")
else:
    # 4. Construction de l'objet GeoJSON personnalisé

    # Création du Feature
    geojson_feature = {
        "type": "Feature",
        "properties": {
            "name": track_name,
            # Le tableau des propriétés de coordonnées
            "coordinateProperties": coord_properties
        },
        "geometry": {
            "type": "LineString",
            "coordinates": coordinates
        }
    }

    # Création du FeatureCollection final
    output_geojson = {
        "type": "FeatureCollection",
        "features": [geojson_feature]
    }

    # 5. Sauvegarde et téléchargement
    output_filename = os.path.splitext(filename)[0] + "_custom.geojson"

    with open(output_filename, 'w') as f:
        # Utilisation de json.dump pour garantir le format GeoJSON
        json.dump(output_geojson, f, indent=2)

    print("\n--- APERÇU DU RÉSULTAT (Structure) ---")
    print(f"Type de géométrie : {output_geojson['features'][0]['geometry']['type']}")
    print(f"Nom de la trace : {track_name}")
    print(f"Nombre de points/propriétés : {len(coordinates)}")

    # Affichage d'un extrait du résultat pour vérification
    print("\n--- Extrait du 'coordinateProperties' ---")
    print(output_geojson['features'][0]['properties']['coordinateProperties'][:2])


    print(f"\n--- TÉLÉCHARGEMENT ---")
    print(f"Le fichier '{output_filename}' est prêt.")
    files.download(output_filename)

--- ÉTAPE 1 : IMPORTATION ---
Veuillez choisir votre fichier .gpx.


Saving Calor Run 20km.gpx to Calor Run 20km.gpx

Traitement du fichier : Calor Run 20km.gpx...

--- APERÇU DU RÉSULTAT (Structure) ---
Type de géométrie : LineString
Nom de la trace : Trace GPX
Nombre de points/propriétés : 872

--- Extrait du 'coordinateProperties' ---
[{'highway': 'track', 'surface': 'asphalt'}, {'highway': 'track', 'surface': 'asphalt'}]

--- TÉLÉCHARGEMENT ---
Le fichier 'Calor Run 20km_custom.geojson' est prêt.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>